<a href="https://colab.research.google.com/github/SchoolIvar/chatbot/blob/main/bolchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask flask-ngrok

In [2]:
%%writefile bolchat.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from flask import Flask, request, jsonify, render_template_string
import re

app = Flask(__name__)

# Simpele “database”
PRODUCTS = {
    "iphone 15":   {"prijs": 1299, "voorraad": 10},
    "samsung galaxy s24": {"prijs": 1099, "voorraad": 5},
    "macbook pro": {"prijs": 2799, "voorraad": 2},
    "sony wh-1000xm4": {"prijs": 299, "voorraad": 20},
}
ORDERS = {
    "12345": "verzonden",
    "67890": "in behandeling",
    "98765": "geleverd",
}

def find_product(name: str):
    name = name.lower()
    results = [f"- {p} (€{d['prijs']}, voorraad: {d['voorraad']})"
               for p, d in PRODUCTS.items() if name in p]
    return "\n".join(results) if results else f"Sorry, ik kon geen producten vinden voor '{name}'."

def track_order(order_id: str):
    status = ORDERS.get(order_id)
    return f"Bestelling {order_id} staat op dit moment in de fase: {status}" if status else f"Sorry, ik kon bestelling {order_id} niet vinden."

def handle_message(text: str):
    text_low = text.lower()

    # intent: productinfo opvragen
    if re.search(r'\b(zoek|product|artikel)\b', text_low):
        m = re.search(r'\b([a-z0-9 ]{2,30})\b', text_low)
        product_name = m.group(1).strip() if m else ""
        return find_product(product_name) if product_name else "Kun je het productspecifiek noemen?"

    # intent: prijs of voorraad
    if re.search(r'\b(prijs|kosten|hoe duur|voorraad)\b', text_low):
        for p in PRODUCTS:
            if p in text_low:
                info = PRODUCTS[p]
                prijs = info["prijs"]
                voorraad = info["voorraad"]
                return f"De {p.title()} kost €{prijs} en er zijn nog {voorraad} op voorraad."
        return "Kun je zeggen van welk product je de prijs bedoelt?"

    # intent: bestelling volgen
    if re.search(r'\b(bestelling|order|volgen|track)\b', text_low):
        m = re.search(r'\b(\d{3,})\b', text_low)
        order_id = m.group(1) if m else ""
        return track_order(order_id) if order_id else "Kun je het ordernummer noemen?"

    # intent: begroeting
    if re.search(r'\b(hallo|goede|hey|hoi)\b', text_low):
        return "Hallo! Waar kan ik je vandaag mee helpen?"

    # intent: afscheid
    if re.search(r'\b(bye|tot ziens|dank je|bedankt)\b', text_low):
        return "Tot ziens! Bedankt voor je bezoek 😊"

    # fallback
    return "Sorry, dat snap ik niet helemaal. Kun je het anders formuleren?"

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/api/message', methods=['POST'])
def api_message():
    data = request.get_json() or {}
    user_msg = data.get('message', '').strip()
    if not user_msg:
        return jsonify({'reply': "Ik heb niets ontvangen."})
    return jsonify({'reply': handle_message(user_msg)})

HTML_TEMPLATE = """
<!doctype html>
<html lang="nl">
<head>
<meta charset="utf-8">
<title>Bol.com Chatbot</title>
<style>
  body{font-family:Arial,sans-serif;background:#f9f9f9;margin:0;padding:0;}
  #chat{border:1px solid #ccc;width:100%;max-width:400px;height:400px;overflow-y:auto;margin:20px auto;background:#fff;padding:10px;}
  #input{width:calc(100% - 60px);padding:5px;}
  #send{width:50px;padding:5px;}
  .msg{margin:5px 0;}
  .bot{color:#333;}
  .user{color:#0066cc;text-align:right;}
  .msg span{display:inline-block;padding:8px 12px;border-radius:12px;}
  .bot span{background:#e1f5fe;}
  .user span{background:#cfe8ff;}
</style>
</head>
<body>
<div id="chat"></div>
<div style="text-align:center;margin:10px;">
  <input id="input" type="text" placeholder="Typ iets...">
  <button id="send">✓</button>
</div>
<script>
const chat=document.getElementById('chat'), input=document.getElementById('input'), sendBtn=document.getElementById('send');
function addMessage(text, sender){const div=document.createElement('div');div.className='msg '+sender;div.innerHTML='<span>'+text+'</span>';chat.appendChild(div);chat.scrollTop=chat.scrollHeight;}
async function sendMessage(){const msg=input.value.trim();if(!msg)return;addMessage(msg,'user');input.value='';try{const res=await fetch('/api/message',{method:'POST',headers:{'Content-Type':'application/json'},body:JSON.stringify({message:msg})});const data=await res.json();addMessage(data.reply,'bot');}catch(e){addMessage('Fout: kan geen verbinding maken.','bot');}}
sendBtn.onclick=sendMessage;input.addEventListener('keypress',e=>{if(e.key==='Enter')sendMessage();});
</script>
</body>
</html>
"""

# Als dit bestand direct wordt uitgevoerd
if __name__ == '__main__':
    from flask_ngrok import run_with_ngrok
    run_with_ngrok(app)
    app.run()


Writing bolchat.py


In [3]:
use_reloader=False

In [4]:
!kill -9 $(lsof -t -i:5000)


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


In [ ]:
!python bolchat.py

 * Serving Flask app 'bolchat'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Usage of ngrok requires a verified account and authtoken.

Sign up for an account: https://dashboard.ngrok.com/signup
Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken

ERR_NGROK_4018

Exception in thread Thread-1:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/pyth